## Employment rates outside of Baltimore NIIF areas: 2017-2021 ACS 5-Year estimates

In [1]:
# From https://www.census.gov/data/developers/data-sets/acs-5year.html
# ACS has non-overlapping datasets that allow comparisons of current ACS data to past ACS data.  
# The 2017-2021 ACS 5-Year estimates can be compared with 2012-2016 ACS 5-Year estimates. 

In [2]:
# see https://towardsdatascience.com/using-the-us-census-api-for-data-analysis-a-beginners-guide-98063791785c
# For API call example, see https://api.census.gov/data/2021/acs/acs5/examples.html

In [3]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

In [4]:
pd.set_option('display.max_rows', 200)
pd.set_option('max_colwidth', 100)

In [5]:
# GIS Analyst, Patrick, provided a csv (created from ARCGIS) that contains a key to match Baltimore's NIIF areas with census tracts

# Store filepath in a variable
NonNIIFs = "./raw_data/NonNIIF_Tracts.csv"

# Read the file with the pandas library
# make sure all of these NIIF census tracts are pulled from the census API (see below)
NonNIIFs_df = pd.read_csv(NonNIIFs)
NonNIIFs_df

,STATEFP,COUNTYFP,TRACTCE,GEOID *,NAME,NAMELSAD
0,24,510,120202,24510120202,1202.02,Census Tract 1202.02
1,24,510,272005,24510272005,2720.05,Census Tract 2720.05
2,24,510,120201,24510120201,1202.01,Census Tract 1202.01
3,24,510,272004,24510272004,2720.04,Census Tract 2720.04
4,24,510,272006,24510272006,2720.06,Census Tract 2720.06
5,24,510,100300,24510100300,1003.00,Census Tract 1003
6,24,510,110100,24510110100,1101.00,Census Tract 1101
7,24,510,110200,24510110200,1102.00,Census Tract 1102
8,24,510,120100,24510120100,1201.00,Census Tract 1201
9,24,510,120600,24510120600,1206.00,Census Tract 1206


In [6]:
# Performing a GET Request and saving the API's response within a variable ("url" is the variable)
# if you get errors, the year you're querying may not be available (response 404 you will get) or there may be a space in your code that you need to remove
# used "\ at end of each line of code and " at beginning of each line so that could use multiple lines of code without causing errors

# Note that all of the census tracts included below were copied from the NonNIIF census tract csv file provided by Patrick/ GIS Analyst

url = "https://api.census.gov/data/2021/acs/acs5?get=NAME,B23025_001E,B23025_002E,B23025_004E,B23025_005E"\
"&for=tract:120202,272005,120201,272004,272006,100300,110100,110200,120100,120600,261100,220100,230300,240100,240200"\
"230100,140100,271400,271501,120700,170100,271503,271101,271102,271200,271300,240300,240400,260800,260900,230200,130600,130700"\
"130803,130804,130805,10200,260700,10100,10300,10400,10500,20100,20200,20300,30100,150500,271900,272003,30200,40100,40200"\
"&in=state:24%20county:510&key=073b804c85c7e45d3710695f0dde8b3c721fdecd"
response = requests.get(url)

response.status_code
#response_json = response.json()
#print(json.dumps(response_json, indent=4, sort_keys=True))

200

In [7]:
#  Get response from census in json format and convert to pandas dataframe
census = response.text
census = json.loads(census)
df = pd.DataFrame.from_dict(census)
df

,0,1,2,3,4,5,6,7
0,NAME,B23025_001E,B23025_002E,B23025_004E,B23025_005E,state,county,tract
1,"Census Tract 1003, Baltimore city, Maryland",2494,0,0,0,24,510,100300
2,"Census Tract 1101, Baltimore city, Maryland",2857,2424,2321,77,24,510,110100
3,"Census Tract 1102, Baltimore city, Maryland",5889,4494,4306,188,24,510,110200
4,"Census Tract 1201, Baltimore city, Maryland",4166,3041,2874,147,24,510,120100
5,"Census Tract 1202.01, Baltimore city, Maryland",1732,1264,1230,27,24,510,120201
6,"Census Tract 1202.02, Baltimore city, Maryland",4990,2186,1915,271,24,510,120202
7,"Census Tract 1206, Baltimore city, Maryland",2564,1511,1464,47,24,510,120600
8,"Census Tract 1207, Baltimore city, Maryland",2067,1645,1500,145,24,510,120700
9,"Census Tract 1306, Baltimore city, Maryland",3430,2797,2756,41,24,510,130600


In [8]:
# use the code below to change the first row of the dataframe into column headers
df.columns = df.iloc[0]
df = df.iloc[1:].reset_index(drop=True)
df

,NAME,B23025_001E,B23025_002E,B23025_004E,B23025_005E,state,county,tract
0,"Census Tract 1003, Baltimore city, Maryland",2494,0,0,0,24,510,100300
1,"Census Tract 1101, Baltimore city, Maryland",2857,2424,2321,77,24,510,110100
2,"Census Tract 1102, Baltimore city, Maryland",5889,4494,4306,188,24,510,110200
3,"Census Tract 1201, Baltimore city, Maryland",4166,3041,2874,147,24,510,120100
4,"Census Tract 1202.01, Baltimore city, Maryland",1732,1264,1230,27,24,510,120201
5,"Census Tract 1202.02, Baltimore city, Maryland",4990,2186,1915,271,24,510,120202
6,"Census Tract 1206, Baltimore city, Maryland",2564,1511,1464,47,24,510,120600
7,"Census Tract 1207, Baltimore city, Maryland",2067,1645,1500,145,24,510,120700
8,"Census Tract 1306, Baltimore city, Maryland",3430,2797,2756,41,24,510,130600
9,"Census Tract 1308.04, Baltimore city, Maryland",1528,1140,1085,55,24,510,130804


In [9]:
import os

In [10]:
os.getcwd()

'C:\\Users\\Jenkir\\Desktop\\BDC\\Baltimore_Together\\measuring-success-metrics\\BT-equity-metric-analysis'

In [11]:
# if need to change the working directory; may need to change the directory for pulling in config file
#os.chdir('C:\\Users\\Jenkir\\Desktop\\BDC\\Baltimore_Together\\measuring-success-metrics\\BT-equity-metric-analysis')


In [12]:
# ACS Table codes to use in cell below;  Universe: Population 16 years and over 
# see https://www.census.gov/programs-surveys/acs/technical-documentation/table-shells.html
# Table B23025: Enployment Status
# B23025_002E	Estimate!!Total:!!In labor force:	B23025_002EA,B23025_002M,B23025_002MA
# B23025_003E	Estimate!!Total:!!In labor force:!!Civilian labor force
# B23025_004E	Estimate!!Total:!!In labor force:!!Civilian labor force:!!Employed	B23025_004EA,B23025_004M,B23025_004MA
# B23025_005E	Estimate!!Total:!!In labor force:!!Civilian labor force:!!Unemployed	B23025_005EA,B23025_005M,B23025_005MA


In [13]:
# rename columns
ACS5_2021 = df.rename(columns={"B23025_001E": "Population age 16 and over",
                               "B23025_002E": "In labor force",
                               "B23025_004E": "# employed, age 16 and over",
                               "B23025_005E": "# unemployed age 16 and over",
                               "NAME": "Name"})
ACS5_2021

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract
0,"Census Tract 1003, Baltimore city, Maryland",2494,0,0,0,24,510,100300
1,"Census Tract 1101, Baltimore city, Maryland",2857,2424,2321,77,24,510,110100
2,"Census Tract 1102, Baltimore city, Maryland",5889,4494,4306,188,24,510,110200
3,"Census Tract 1201, Baltimore city, Maryland",4166,3041,2874,147,24,510,120100
4,"Census Tract 1202.01, Baltimore city, Maryland",1732,1264,1230,27,24,510,120201
5,"Census Tract 1202.02, Baltimore city, Maryland",4990,2186,1915,271,24,510,120202
6,"Census Tract 1206, Baltimore city, Maryland",2564,1511,1464,47,24,510,120600
7,"Census Tract 1207, Baltimore city, Maryland",2067,1645,1500,145,24,510,120700
8,"Census Tract 1306, Baltimore city, Maryland",3430,2797,2756,41,24,510,130600
9,"Census Tract 1308.04, Baltimore city, Maryland",1528,1140,1085,55,24,510,130804


In [14]:
ACS5_2021_final = ACS5_2021.copy()
ACS5_2021_final.head()

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract
0,"Census Tract 1003, Baltimore city, Maryland",2494,0,0,0,24,510,100300
1,"Census Tract 1101, Baltimore city, Maryland",2857,2424,2321,77,24,510,110100
2,"Census Tract 1102, Baltimore city, Maryland",5889,4494,4306,188,24,510,110200
3,"Census Tract 1201, Baltimore city, Maryland",4166,3041,2874,147,24,510,120100
4,"Census Tract 1202.01, Baltimore city, Maryland",1732,1264,1230,27,24,510,120201


In [15]:
# Add in employment rate (# employed, age 16 and over / Population age 16 and over)
ACS5_2021_final["Employment rate"] = 100 * \
    ACS5_2021_final["# employed, age 16 and over"].astype(
        int) / ACS5_2021_final["Population age 16 and over"].astype(int)
ACS5_2021_final

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract,Employment rate
0,"Census Tract 1003, Baltimore city, Maryland",2494,0,0,0,24,510,100300,0.000000
1,"Census Tract 1101, Baltimore city, Maryland",2857,2424,2321,77,24,510,110100,81.239062
2,"Census Tract 1102, Baltimore city, Maryland",5889,4494,4306,188,24,510,110200,73.119375
3,"Census Tract 1201, Baltimore city, Maryland",4166,3041,2874,147,24,510,120100,68.987038
4,"Census Tract 1202.01, Baltimore city, Maryland",1732,1264,1230,27,24,510,120201,71.016166
5,"Census Tract 1202.02, Baltimore city, Maryland",4990,2186,1915,271,24,510,120202,38.376754
6,"Census Tract 1206, Baltimore city, Maryland",2564,1511,1464,47,24,510,120600,57.098284
7,"Census Tract 1207, Baltimore city, Maryland",2067,1645,1500,145,24,510,120700,72.568940
8,"Census Tract 1306, Baltimore city, Maryland",3430,2797,2756,41,24,510,130600,80.349854
9,"Census Tract 1308.04, Baltimore city, Maryland",1528,1140,1085,55,24,510,130804,71.007853


In [16]:
# Round rate to one decimal point
ACS5_2021_final["Employment rate"] = ACS5_2021_final["Employment rate"].astype(float).round(1)
ACS5_2021_final

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract,Employment rate
0,"Census Tract 1003, Baltimore city, Maryland",2494,0,0,0,24,510,100300,0.0
1,"Census Tract 1101, Baltimore city, Maryland",2857,2424,2321,77,24,510,110100,81.2
2,"Census Tract 1102, Baltimore city, Maryland",5889,4494,4306,188,24,510,110200,73.1
3,"Census Tract 1201, Baltimore city, Maryland",4166,3041,2874,147,24,510,120100,69.0
4,"Census Tract 1202.01, Baltimore city, Maryland",1732,1264,1230,27,24,510,120201,71.0
5,"Census Tract 1202.02, Baltimore city, Maryland",4990,2186,1915,271,24,510,120202,38.4
6,"Census Tract 1206, Baltimore city, Maryland",2564,1511,1464,47,24,510,120600,57.1
7,"Census Tract 1207, Baltimore city, Maryland",2067,1645,1500,145,24,510,120700,72.6
8,"Census Tract 1306, Baltimore city, Maryland",3430,2797,2756,41,24,510,130600,80.3
9,"Census Tract 1308.04, Baltimore city, Maryland",1528,1140,1085,55,24,510,130804,71.0


In [17]:
# Add in labor force participation rate (In labor force / Population age 16 and over)
ACS5_2021_final["Labor force participation rate"] = 100 * \
    ACS5_2021_final["In labor force"].astype(
        int) / ACS5_2021_final["Population age 16 and over"].astype(int)
ACS5_2021_final

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract,Employment rate,Labor force participation rate
0,"Census Tract 1003, Baltimore city, Maryland",2494,0,0,0,24,510,100300,0.0,0.000000
1,"Census Tract 1101, Baltimore city, Maryland",2857,2424,2321,77,24,510,110100,81.2,84.844242
2,"Census Tract 1102, Baltimore city, Maryland",5889,4494,4306,188,24,510,110200,73.1,76.311768
3,"Census Tract 1201, Baltimore city, Maryland",4166,3041,2874,147,24,510,120100,69.0,72.995679
4,"Census Tract 1202.01, Baltimore city, Maryland",1732,1264,1230,27,24,510,120201,71.0,72.979215
5,"Census Tract 1202.02, Baltimore city, Maryland",4990,2186,1915,271,24,510,120202,38.4,43.807615
6,"Census Tract 1206, Baltimore city, Maryland",2564,1511,1464,47,24,510,120600,57.1,58.931357
7,"Census Tract 1207, Baltimore city, Maryland",2067,1645,1500,145,24,510,120700,72.6,79.583938
8,"Census Tract 1306, Baltimore city, Maryland",3430,2797,2756,41,24,510,130600,80.3,81.545190
9,"Census Tract 1308.04, Baltimore city, Maryland",1528,1140,1085,55,24,510,130804,71.0,74.607330


In [18]:
# Round rate to one decimal point
ACS5_2021_final["Labor force participation rate"] = ACS5_2021_final["Labor force participation rate"].astype(float).round(1)
ACS5_2021_final

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract,Employment rate,Labor force participation rate
0,"Census Tract 1003, Baltimore city, Maryland",2494,0,0,0,24,510,100300,0.0,0.0
1,"Census Tract 1101, Baltimore city, Maryland",2857,2424,2321,77,24,510,110100,81.2,84.8
2,"Census Tract 1102, Baltimore city, Maryland",5889,4494,4306,188,24,510,110200,73.1,76.3
3,"Census Tract 1201, Baltimore city, Maryland",4166,3041,2874,147,24,510,120100,69.0,73.0
4,"Census Tract 1202.01, Baltimore city, Maryland",1732,1264,1230,27,24,510,120201,71.0,73.0
5,"Census Tract 1202.02, Baltimore city, Maryland",4990,2186,1915,271,24,510,120202,38.4,43.8
6,"Census Tract 1206, Baltimore city, Maryland",2564,1511,1464,47,24,510,120600,57.1,58.9
7,"Census Tract 1207, Baltimore city, Maryland",2067,1645,1500,145,24,510,120700,72.6,79.6
8,"Census Tract 1306, Baltimore city, Maryland",3430,2797,2756,41,24,510,130600,80.3,81.5
9,"Census Tract 1308.04, Baltimore city, Maryland",1528,1140,1085,55,24,510,130804,71.0,74.6


In [19]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract code
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
ACS5_2021_final["GEOID"] = ACS5_2021_final['state'] + ACS5_2021_final['county'] + ACS5_2021_final['tract']
ACS5_2021_final

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract,Employment rate,Labor force participation rate,GEOID
0,"Census Tract 1003, Baltimore city, Maryland",2494,0,0,0,24,510,100300,0.0,0.0,24510100300
1,"Census Tract 1101, Baltimore city, Maryland",2857,2424,2321,77,24,510,110100,81.2,84.8,24510110100
2,"Census Tract 1102, Baltimore city, Maryland",5889,4494,4306,188,24,510,110200,73.1,76.3,24510110200
3,"Census Tract 1201, Baltimore city, Maryland",4166,3041,2874,147,24,510,120100,69.0,73.0,24510120100
4,"Census Tract 1202.01, Baltimore city, Maryland",1732,1264,1230,27,24,510,120201,71.0,73.0,24510120201
5,"Census Tract 1202.02, Baltimore city, Maryland",4990,2186,1915,271,24,510,120202,38.4,43.8,24510120202
6,"Census Tract 1206, Baltimore city, Maryland",2564,1511,1464,47,24,510,120600,57.1,58.9,24510120600
7,"Census Tract 1207, Baltimore city, Maryland",2067,1645,1500,145,24,510,120700,72.6,79.6,24510120700
8,"Census Tract 1306, Baltimore city, Maryland",3430,2797,2756,41,24,510,130600,80.3,81.5,24510130600
9,"Census Tract 1308.04, Baltimore city, Maryland",1528,1140,1085,55,24,510,130804,71.0,74.6,24510130804


In [20]:
# Remove extraneous columns
ACS5_2021_final = ACS5_2021_final.drop(['Name','state','county','tract'], axis=1)
ACS5_2021_final

,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,Employment rate,Labor force participation rate,GEOID
0,2494,0,0,0,0.0,0.0,24510100300
1,2857,2424,2321,77,81.2,84.8,24510110100
2,5889,4494,4306,188,73.1,76.3,24510110200
3,4166,3041,2874,147,69.0,73.0,24510120100
4,1732,1264,1230,27,71.0,73.0,24510120201
5,4990,2186,1915,271,38.4,43.8,24510120202
6,2564,1511,1464,47,57.1,58.9,24510120600
7,2067,1645,1500,145,72.6,79.6,24510120700
8,3430,2797,2756,41,80.3,81.5,24510130600
9,1528,1140,1085,55,71.0,74.6,24510130804


In [21]:
# change order of columns
ACS5_2021_final = ACS5_2021_final[['GEOID', 'Population age 16 and over', '# employed, age 16 and over', 
                                   '# unemployed age 16 and over',"Employment rate", 
                                   'In labor force',"Labor force participation rate"]]
ACS5_2021_final 

,GEOID,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,Employment rate,In labor force,Labor force participation rate
0,24510100300,2494,0,0,0.0,0,0.0
1,24510110100,2857,2321,77,81.2,2424,84.8
2,24510110200,5889,4306,188,73.1,4494,76.3
3,24510120100,4166,2874,147,69.0,3041,73.0
4,24510120201,1732,1230,27,71.0,1264,73.0
5,24510120202,4990,1915,271,38.4,2186,43.8
6,24510120600,2564,1464,47,57.1,1511,58.9
7,24510120700,2067,1500,145,72.6,1645,79.6
8,24510130600,3430,2756,41,80.3,2797,81.5
9,24510130804,1528,1085,55,71.0,1140,74.6


In [22]:
# Prepare to create df where we will sum the population, # employed, # unemployed, in labor force for all of the NIIF tracts 
ACS5_2021_sum1 = ACS5_2021_final.copy()
ACS5_2021_sum1

,GEOID,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,Employment rate,In labor force,Labor force participation rate
0,24510100300,2494,0,0,0.0,0,0.0
1,24510110100,2857,2321,77,81.2,2424,84.8
2,24510110200,5889,4306,188,73.1,4494,76.3
3,24510120100,4166,2874,147,69.0,3041,73.0
4,24510120201,1732,1230,27,71.0,1264,73.0
5,24510120202,4990,1915,271,38.4,2186,43.8
6,24510120600,2564,1464,47,57.1,1511,58.9
7,24510120700,2067,1500,145,72.6,1645,79.6
8,24510130600,3430,2756,41,80.3,2797,81.5
9,24510130804,1528,1085,55,71.0,1140,74.6


In [23]:
# Remove extraneous columns
ACS5_2021_sum = ACS5_2021_sum1.drop(['Employment rate','Labor force participation rate'], axis=1)
ACS5_2021_sum.dtypes

0
GEOID                           object
Population age 16 and over      object
# employed, age 16 and over     object
# unemployed age 16 and over    object
In labor force                  object
dtype: object

In [24]:
# change data types from "object"/ text to "float"/numeric
ACS5_2021_sum[['Population age 16 and over', '# employed, age 16 and over', '# unemployed age 16 and over',
               'In labor force']]= ACS5_2021_sum[['Population age 16 and over', '# employed, age 16 and over', '# unemployed age 16 and over',
               'In labor force']].astype(float)
ACS5_2021_sum.dtypes

0
GEOID                            object
Population age 16 and over      float64
# employed, age 16 and over     float64
# unemployed age 16 and over    float64
In labor force                  float64
dtype: object

In [25]:
#  sum the population age 16 and over and # employed , age 16 and over for all of the census tracts that are in NIIF boundary
# need to make sure the data types are numeric and not "object" first
# "s" is a Pandas series
s = ACS5_2021_sum[['Population age 16 and over', '# employed, age 16 and over', '# unemployed age 16 and over',
               'In labor force']].sum()

In [26]:
# Convert the series ("s") to a dataframe
sums2 = s.to_frame()
sums2

,0
0,
Population age 16 and over,97877.0
"# employed, age 16 and over",61283.0
# unemployed age 16 and over,2936.0
In labor force,64384.0


In [27]:
# "Flip" the df so we have column headers
sums = sums2.transpose()
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force
0,97877.0,61283.0,2936.0,64384.0


In [28]:
# Add in employment rate (# employed, age 16 and over / Population age 16 and over)
sums["Employment rate"] = 100 * \
    sums["# employed, age 16 and over"].astype(
        int) / sums["Population age 16 and over"].astype(int)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate
0,97877.0,61283.0,2936.0,64384.0,62.612258


In [29]:
# Round rate to one decimal point
sums["Employment rate"] = sums["Employment rate"].astype(float).round(1)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate
0,97877.0,61283.0,2936.0,64384.0,62.6


In [30]:
# Add in labor force participation rate (In labor force / Population age 16 and over)
sums["Labor force participation rate"] = 100 * \
    sums["In labor force"].astype(
        int) / sums["Population age 16 and over"].astype(int)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate,Labor force participation rate
0,97877.0,61283.0,2936.0,64384.0,62.6,65.78052


In [31]:
# Round rate to one decimal point
sums["Labor force participation rate"] = sums["Labor force participation rate"].astype(float).round(1)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate,Labor force participation rate
0,97877.0,61283.0,2936.0,64384.0,62.6,65.8


In [32]:
# Add in unemployment rate (# unemployed, age 16 and over / Population age 16 and over)
sums["Unemployment rate"] = 100 * \
    sums["# unemployed age 16 and over"].astype(
        int) / sums["Population age 16 and over"].astype(int)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate,Labor force participation rate,Unemployment rate
0,97877.0,61283.0,2936.0,64384.0,62.6,65.8,2.999683


In [33]:
# Round rate to one decimal point
sums["Unemployment rate"] = sums["Unemployment rate"].astype(float).round(1)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate,Labor force participation rate,Unemployment rate
0,97877.0,61283.0,2936.0,64384.0,62.6,65.8,3.0


In [34]:
# Export file as a CSV, without the Pandas index, but with the header
# Do not run this last code block until you have all previous code blocks in their final form:

sums.to_csv("Jupyter_nb_final_csvs/NonNIIF_employment_2017-2021.csv", index = False, header=True)